In [1]:
import pygame, sys
import os
import numpy as np
import itertools
import random
import time

_image_library = {}
def highlight(screen, color, x,y):
    if (x//150 < map_size) and (y//150 < map_size):
        pygame.draw.rect(screen, color, [x//150*150, y//150*150, 160, 160], 0)
def get_image(path):
        global _image_library
        image = _image_library.get(path)
        if image == None:
                canonicalized_path = path.replace('/', os.sep).replace('\\', os.sep)
                image = pygame.image.load(canonicalized_path)
                _image_library[path] = image
        return image
def init_political(player_num, map_size):
    political_map = np.random.randint(0,player_num,(map_size,map_size))
    return political_map
def init_army(map_size):
    army_map = np.random.randint(1,7,(map_size,map_size))
    return army_map
def draw_board(screen, political_map, army_map):
    for y,x in itertools.product(range(political_map.shape[0]), range(political_map.shape[1])):
        w = 150-10
        h = 150-10
        tile_color = color[political_map[y][x]]
        pygame.draw.rect(screen, tile_color, pygame.Rect(10+x*150, 10+y*150, w, h))
        army_size = army_map[y][x]
        screen.blit(get_image('images/samurai.png'), (x*150+50, y*150+30))
        # army number text
        font = pygame.font.SysFont("comicsansms", 30)
        text = str(army_map[y][x])
        text = font.render(text, True, (255,255,255))
        screen.blit(text,(x*150+67, y*150+50))
def verify_move(attacker, defender, player, phase):
    ay = int(attacker[0])
    ax = int(attacker[1])
    dy = int(defender[0])
    dx = int(defender[1])
    if (ax > map_size) or (ay > map_size) or (dx > map_size) or (dy > map_size):
        #print("chosen location invalid")
        return 0
    if (political_map[ay][ax] != player) and (phase == "attack"):
        print(ay,ax)
        #print(political_map[ay][ax])
        print("Start location does not belong to you")
        return 0
    if (political_map[dy][dx] == player) and (phase == "defend"):
        #print(dy,dx)
        #print(political_map[dy][dx])
        print("You cannot move towards own region")
        return 0
    if ((abs(dy-ay)>1) or (abs(ax-dx)>1)) and (phase == "defend"):
        print("Please only attack a hostile adjacent region")
        return 0
    else:
        return 1
def battle(political_map, army_map, attacker, defender, current):
    ay = int(attacker[0])
    ax = int(attacker[1])
    dy = int(defender[0])
    dx = int(defender[1])
    if (ax != dx) & (ay !=dy):
        print("Invalid move. You can only attack adjacent regions")
        return (political_map, army_map, 0)
    attack_army = army_map[ay][ax]
    defend_army = army_map[dy][dx]
    attack_power = 0
    defend_power = 0
    for i in range(attack_army):
        power = random.randint(1, 6)
        attack_power = attack_power + power
        display_battle(political_map[ay][ax], 'attacker', i, power)
    # display attack power text
    font = pygame.font.SysFont("comicsansms", 30)
    text = str(attack_power)
    text = font.render(text, True, color[current])
    screen.blit(text,(810, 10))
    
    for i in range(defend_army):
        power = random.randint(1, 6)
        defend_power = defend_power + power
        display_battle(political_map[dy][dx], 'defender', i, power)
    # display attack power text
    font = pygame.font.SysFont("comicsansms", 30)
    text = str(defend_power)
    text = font.render(text, True, color[political_map[dy][dx]])
    screen.blit(text,(910, 10))
    
    text = 'Player '+ str(current)
    print(text , " attacked ", defend_army, "defending armies with ", attack_army, "armies")
    print(text , "'s attack power is ", attack_power, "and the enemy defend power is", defend_power)
    if attack_power > defend_power:
        print(text , " overpowered the defender and successfully captured region ", dy, dx)
        political_map[dy][dx] = current
        army_map[dy][dx] = attack_army - 1
        army_map[ay][ax] = 1
    else:
        print(text, " failed to capture location ", dy, dx, "and lost half the troops")
        army_map[ay][ax] = int(attack_army/2)
    return (political_map, army_map, 1)
def display_battle(player, role, army, power):
    
    dice = ['images/red1.png','images/red2.png','images/red3.png','images/red4.png','images/red5.png','images/red6.png',
           'images/green1.png','images/green2.png','images/green3.png','images/green4.png','images/green5.png','images/green6.png',
           'images/blue1.png','images/blue2.png','images/blue3.png','images/blue4.png','images/blue5.png','images/blue6.png',
           'images/cyan1.png','images/cyan2.png','images/cyan3.png','images/cyan4.png','images/cyan5.png','images/cyan6.png']
    if role == 'attacker':
        x = 800
    else:
        x = 900
    screen.blit(get_image(dice[player * 6 + power - 1]), (x, 100*army+50))
def recruit(political_map, current):
    current_count = 0
    for i in range(map_size):
        for j in range(map_size):
            if political_map[i][j]==current:
                current_count = current_count + 1
    return current_count
x = 80
y = 80
width = 1200
height = 800
color = ((200, 50, 50), (50, 200, 50), (50, 50, 200), (50,200,200))#red, green, blue, cyan
pygame.init()
screen = pygame.display.set_mode((width, height))
done = False
player_num = 2
map_size = 5
political_map = init_political(player_num, map_size)

army_map = init_army(map_size)
phase = "attack"
player = 0
#initialize text
font = pygame.font.SysFont("comicsansms", 30)
text = "Game begins!"
master_text = font.render(text, True, (255,255,255))
attacker_selected = False
defender_selected = False

while not done:
        
        for event in pygame.event.get():
                if event.type == pygame.QUIT:
                        done = True
                if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                        done = True

        pressed = pygame.key.get_pressed()
        mousePressed = pygame.mouse.get_pressed()
        if mousePressed:
            x = pygame.mouse.get_pos()[0]
            y = pygame.mouse.get_pos()[1]
            
        if pressed[pygame.K_p]: 
            player += 1
            player = player % player_num
            print("Player\n", str(player))
            phase = "deploy"
            recruits = recruit(political_map, player)
            deployed = False
            # print
            font = pygame.font.SysFont("comicsansms", 20)
            text = "Choose one of your armies to deploy " + str(recruits) + " recruits."
            master_text = font.render(text, True, (255,255,255))
            

        screen.fill((50, 50, 50), pygame.Rect(0,0,800-40,800))
        screen.blit(master_text,(200, 750)) 
        
        # update screen after this line
        highlight_color = [255,255,255]
        highlight(screen,highlight_color, x,y)
        font = pygame.font.SysFont("comicsansms", 20)
        text = "Player "+ str(player) + "'s turn."
        prompt = font.render(text, True, color[player])
        screen.blit(prompt,(10, 750))
                
        draw_board(screen, political_map, army_map)
        #screen.blit(get_image('images/cursor.png'), (x, y))
        #if pressed[pygame.K_SPACE]:#selected
        
        if mousePressed[0] and (x//150 < map_size) and (y//150 < map_size):
            print(x,y)
            print(phase)
            select_x = x//150
            select_y = y//150
            if phase == "deploy":
                while deployed == False:
                    if political_map[select_y][select_x]==player:
                        army_map[select_y][select_x] = army_map[select_y][select_x] + recruits
                        deployed = True
                        phase = "attack"
                    
            elif phase == "attack":
                attacker = [select_y, select_x]
                if verify_move(attacker, [0,0], player, phase)==1:
                    
                    phase = "defend"
                    attacker_selected = True
                    # print
                    font = pygame.font.SysFont("comicsansms", 20)
                    text = "Army selected"
                    master_text = font.render(text, True, (255,255,255))
                else:
                    # print
                    font = pygame.font.SysFont("comicsansms", 20)
                    text = "Please select your army again."
                    master_text = font.render(text, True, (255,255,255))
                    
            elif phase == "defend":
                defender = [select_y, select_x]
                if verify_move(attacker, defender, player, phase)==1:
                    highlight_color = color[player]
                    highlight(screen,highlight_color, x,y)
                    # print
                    font = pygame.font.SysFont("comicsansms", 20)
                    text = "Attack executed."
                    master_text = font.render(text, True, (255,255,255))
                    defender_selected = True
                else:
                    # print
                    font = pygame.font.SysFont("comicsansms", 20)
                    text = "Please select enemy army again."
                    master_text = font.render(text, True, (255,255,255))
                    
        screen.blit(master_text,(200, 750))        
        if (attacker_selected == True) and (defender_selected == True):
            screen.fill((0, 0, 0), pygame.Rect(800,0,300,800))
            (political_map, army_map, result) = battle(political_map, army_map, attacker, defender, player)
            if result == 1:
                attacker_selected = False
                defender_selected = False
                phase = "attack"
            else:
                # print
                font = pygame.font.SysFont("comicsansms", 20)
                text = "Please select enemy army again."
                master_text = font.render(text, True, (255,255,255))
                defender_selected = False
        pygame.display.flip()
pygame.quit()
quit()

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
673 500
attack
673 500
defend
You cannot move towards own region
562 515
defend
Player 0  attacked  3 defending armies with  6 armies
Player 0 's attack power is  27 and the enemy defend power is 10
Player 0  overpowered the defender and successfully captured region  3 3
545 522
attack
546 428
defend
Player 0  attacked  2 defending armies with  5 armies
Player 0 's attack power is  20 and the enemy defend power is 5
Player 0  overpowered the defender and successfully captured region  2 3
546 428
attack
684 414
defend
Player 0  attacked  3 defending armies with  4 armies
Player 0 's attack power is  14 and the enemy defend power is 8
Player 0  overpowered the defender and successfully captured region  2 4
671 244
attack
667 126
defend
Player 0  attacked  1 defending armies with  4 armies
Player 0 's attack power is  21 and the enemy defend power is 6
Player 0  overpowered the defender and successfully c

107 433
attack
76 511
defend
Player 0  attacked  1 defending armies with  1 armies
Player 0 's attack power is  5 and the enemy defend power is 1
Player 0  overpowered the defender and successfully captured region  3 0
132 415
attack
98 417
defend
You cannot move towards own region
191 489
defend
Invalid move. You can only attack adjacent regions
397 483
defend
Please only attack a hostile adjacent region
704 111
defend
You cannot move towards own region
677 200
defend
Please only attack a hostile adjacent region
Player
 1
106 68
deploy
106 68
attack
249 65
defend
Player 1  attacked  1 defending armies with  14 armies
Player 1 's attack power is  60 and the enemy defend power is 1
Player 1  overpowered the defender and successfully captured region  0 1
249 65
attack
384 108
defend
Player 1  attacked  1 defending armies with  13 armies
Player 1 's attack power is  47 and the enemy defend power is 2
Player 1  overpowered the defender and successfully captured region  0 2
384 108
attack
4